In [1]:
import pandas as pd

elec_df = pd.read_csv('testfiles/utility_rates/Average_retail_price_of_electricity.csv')
ng_df = pd.read_csv('testfiles/utility_rates/Average_retail_price_of_natural_gas.csv')
oil_df = pd.read_csv('testfiles/utility_rates/Average_retail_price_of_fuel_oil.csv')
prop_df = pd.read_csv('testfiles/utility_rates/Average_retail_price_of_propane.csv')

In [2]:
ng_df['Short State'] = ng_df['Sourcekey'].astype(str).str[-3:-1]
ng_df['Long State'] = ng_df['Date'].str.split(' Price ',expand=True)[0]
ng_df.drop(columns=['Sourcekey', 'Date'], inplace=True)

oil_df['Short State'] = oil_df['Sourcekey'].astype(str).str[-6:-4]
oil_df['Long State'] = oil_df['Date'].str.split(' No. ',expand=True)[0].str.split('Weekly ',expand=True)[1]
oil_df.drop(columns=['Sourcekey', 'Date'], inplace=True)

prop_df['Short State'] = prop_df['Sourcekey'].astype(str).str[-6:-4]
prop_df['Long State'] = prop_df['Date'].str.split(' Propane ',expand=True)[0].str.split('Weekly ',expand=True)[1]
prop_df.drop(columns=['Sourcekey', 'Date'], inplace=True)
prop_df


,Average Propane Dollars per Gallon,Short State,Long State
0,3.679875,CT,Connecticut
1,3.126406,ME,Maine
2,3.509344,MA,Massachusetts
3,3.459125,NH,New Hampshire
4,3.510625,RI,Rhode Island
5,3.446219,VT,Vermont
6,3.590031,DE,Delaware
7,3.463250,MD,Maryland
8,3.284344,NJ,New Jersey
9,3.305438,NY,New York


In [3]:
elec_ng_df = elec_df.merge(ng_df, how='outer', left_on='State', right_on='Long State')
elec_ng_df.drop(columns=['State'], inplace=True)
elec_ng_oil_df = elec_ng_df.merge(oil_df, how='outer', on='Short State')
elec_ng_oil_df.drop(columns=['Long State_x', 'Long State_y'], inplace=True)
full_df = elec_ng_oil_df.merge(prop_df, how='outer', on='Short State')
full_df.drop(columns=['Long State'], inplace=True)

full_df.rename(columns={'Short State': 'State'}, inplace=True)
full_df.set_index('State', inplace=True)

# Replace all states where we don't have data with the average of all states where we do
full_df.fillna(full_df.mean(), inplace=True)
full_df['Average Natural Gas $/therm'] = full_df['Average Natural Gas $/thousand Cubic Feet'] / 10.37 # Therm Factor
full_df['Average Fuel Oil $/therm'] = full_df['Average Fuel Oil Dollars per gallon'] / 1.4 # Therms per gallon
full_df['Average Propane $/therm'] = full_df['Average Propane Dollars per Gallon'] / .915 # Therms per gallon
full_df.drop(['Average Natural Gas $/thousand Cubic Feet','Average Fuel Oil Dollars per gallon','Average Propane Dollars per Gallon'], inplace=True, axis=1)
# melted_df = full_df.reset_index().melt(id_vars=['State'])
full_df.to_parquet('utility_rates.parquet')
full_df

,Average Electricity Dollars per kWh,Average Natural Gas $/therm,Average Fuel Oil $/therm,Average Propane $/therm
State,,,,
AK,0.245542,1.151519,2.753275,2.964062
AL,0.149008,2.136853,2.753275,3.511988
AR,0.122658,1.811496,2.753275,2.450615
AZ,0.146658,2.164537,2.753275,2.964062
CA,0.310325,1.846581,2.753275,2.964062
CO,0.146950,1.301953,2.753275,2.443272
CT,0.280300,2.032242,2.784710,4.021721
DC,0.172125,1.890268,2.753275,2.964062
DE,0.164733,2.013099,3.102768,3.923531


In [4]:
elec_df

,State,Average Electricity Dollars per kWh
0,Connecticut,0.280300
1,Maine,0.247367
2,Massachusetts,0.286325
3,New Hampshire,0.239592
4,Rhode Island,0.291192
5,Vermont,0.214717
6,New Jersey,0.183642
7,New York,0.233825
8,Pennsylvania,0.177650
9,Illinois,0.155892
